# Mushroom
http://archive.ics.uci.edu/ml/datasets/Mushroom
- 8124종류의 버섯의 특징과 독의 유무를 정리해 놓은 데이터셋
- 버섯 모양의 특징을 기반으로 독의 유무를 판단

한 줄(row)이 버섯의 한 종류.
1번째 컬럼은 독의 유무 : 독(p) , 식물 (e)
2번쨰 컬럼은 버섯의 모리모양 : b(벨 형태), c(원뿔), x(볼록), f(평평), k(혹), s(오목)
4번째 컬럼은 버섯의 머리색 : n(갈색), b(황갈색), c(연한 갈색), g(회색)

- 자세한 내용은 위의 사이트에서 확인

- 인공신경망, 의사결정나무, 랜덤포레스트, SVM, 로지스틱 회귀분석 

In [2]:
options(warn = -1)
library(caret)
library(nnet)
library(e1071)
library(randomForest)
library(party)


library(dplyr)

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich

Attaching package: ‘dplyr’

The following object is masked from ‘package:randomForest’:

    combine

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, 

### archive.ics.uci.edu/ml/datasets/Mushroom > 데이터셋 많음
- 8024종류의 버섯의 특징과 독의 유무를 정리한 데이터셋
- 버섯 모양의 특징을 기반으로 독의 유무를 판단

한 줄(row)이 버섯 한 종류
- p 컬럼(1): p - poison, e - eatable
- x 컬럼(2): 버섯의 머리모양 b: 벨형태, c: 원뿔형태, x: 볼록형태, f: 평평, k: 혹, s: 오목
- n 컬럼(4): 버섯의 머리색 n(갈색), b(황갈색), c(연한갈색), g(회색)

* 자세한 내용은 위의 사이트에서 확인

In [2]:
IGN <- function(data.tr, x, y){
  model <- nnet(V1 ~ . , data = data.tr, size=3 )
  pred <- predict(model, x, type = "class")
  return(mean(pred == y))
}

In [3]:
DMT <- function(data.tr, x, y){
  model <- ctree(V1 ~ . , data = data.tr )
  pred <- predict(model, x)
  return(mean(pred == y))
}

In [4]:
RF <- function(data.tr, x, y){
  model <- randomForest(V1 ~ . , data = data.tr, ntree = 100 )
  pred <- predict(model, x)
  return(mean(pred == y))
}


In [5]:
SVM <- function(data.tr, x, y){
  model = svm(V1 ~ . , data=data.tr)
  pred <- predict(model, x)
  return(mean(pred == y))
}


In [6]:
MRR <- function(data.tr, x, y){
  model = multinom(V1 ~ . , data=data.tr)
  pred <- predict(model, x)
  return(mean(pred == y))
}

In [10]:
반복수행 <- function(n){

  m <- read.csv('../Data/mushroom.csv', header=F)[, -17]

  samp <- createDataPartition(m$V1, p=0.7, list=F)

  # 학습용 데이터 만들기
  data.tr <- m[samp,]
  # 검증용 데이터 만들기
  data.test <- m[-samp,]
  # 문제 및 정답 만들기
  x <- subset(data.test, select = -V1)
  y <- data.test$V1


  ignS = 0
  dmtS = 0
  rfS = 0
  svmS = 0
  mrrS = 0

  for(i in 1:n){
    ignS = ignS + IGN(data.tr, x, y)
    dmtS = dmtS + DMT(data.tr, x, y)
    rfS = rfS + RF(data.tr, x, y)
    svmS = svmS + SVM(data.tr, x, y)
    mrrS = mrrS + MRR(data.tr, x, y)
  }

  예측력 <- c(ignS, dmtS, rfS, svmS, mrrS)
  예측력 <- round(예측력 / n , 8)
  result <- data.frame(예측력)
  rownames(result) = c('인공신경망', '의사결정나무', 'RandmForest', 'SVM', '로지스틱회귀분석')

  return(result)
}

In [11]:
반복수행(5)


# weights:  292
initial  value 3830.060632 
iter  10 value 445.910451
iter  20 value 155.250497
iter  30 value 108.791286
iter  40 value 77.695082
iter  50 value 21.385897
iter  60 value 15.408340
iter  70 value 13.262888
iter  80 value 13.212786
iter  90 value 12.594686
iter 100 value 0.020628
final  value 0.020628 
stopped after 100 iterations
# weights:  97 (96 variable)
initial  value 3942.621163 
iter  10 value 147.193626
iter  20 value 15.289145
iter  30 value 1.591655
iter  40 value 0.591773
iter  50 value 0.216603
iter  60 value 0.067702
iter  70 value 0.022862
iter  80 value 0.011356
iter  90 value 0.002589
iter 100 value 0.001715
final  value 0.001715 
stopped after 100 iterations
# weights:  292
initial  value 3883.103236 
iter  10 value 210.586389
iter  20 value 34.951549
iter  30 value 22.003600
iter  40 value 0.050388
iter  50 value 0.000178
final  value 0.000098 
converged
# weights:  97 (96 variable)
initial  value 3942.621163 
iter  10 value 147.193626
iter  20 value 1

,예측력
인공신경망,0.9943350
의사결정나무,0.9979474
RandmForest,1.0000000
SVM,0.9946634
로지스틱회귀분석,1.0000000


In [13]:
m <- read.csv('../Data/mushroom.csv', header=F)[, -17]

samp <- createDataPartition(m$V1, p=0.7, list=F)

# 학습용 데이터 만들기
data.tr <- m[samp,]
# 검증용 데이터 만들기
data.test <- m[-samp,]
# 문제 및 정답 만들기
x <- subset(data.test, select = -V1)
y <- data.test$V1
ignS = 0
dmtS = 0
rfS = 0
svmS = 0
mrrS = 0

for(i in 1:5){
  ignS = ignS + IGN(data.tr, x, y)
  dmtS = dmtS + DMT(data.tr, x, y)
  rfS = rfS + RF(data.tr, x, y)
  svmS = svmS + SVM(data.tr, x, y)
  mrrS = mrrS + MRR(data.tr, x, y)

  cat(ignS , dmtS , rfS , svmS, mrrS)
}

# weights:  292
initial  value 4011.357250 
iter  10 value 676.746936
iter  20 value 47.909277
iter  30 value 0.519085
iter  40 value 0.021931
iter  50 value 0.003386
iter  60 value 0.000538
iter  70 value 0.000164
iter  80 value 0.000159
final  value 0.000087 
converged
# weights:  97 (96 variable)
initial  value 3942.621163 
iter  10 value 138.301992
iter  20 value 16.496836
iter  30 value 1.416622
iter  40 value 0.524633
iter  50 value 0.155748
iter  60 value 0.057190
iter  70 value 0.015528
iter  80 value 0.008608
iter  90 value 0.001606
iter 100 value 0.001075
final  value 0.001075 
stopped after 100 iterations
1 0.9987685 1 0.9971264 1# weights:  292
initial  value 3961.694133 
iter  10 value 364.880774
iter  20 value 231.301397
iter  30 value 17.580511
iter  40 value 0.543402
iter  50 value 0.116868
iter  60 value 0.008757
iter  70 value 0.003402
iter  80 value 0.000243
iter  90 value 0.000243
iter 100 value 0.000234
final  value 0.000234 
stopped after 100 iterations
# weights:

In [4]:
m <- read.csv('../Data/mushroom.csv', header=F)
str(m) # 전부 팩터 ,
m <- arrange(m , m$V1) 
summary(m) # e,p의 갯수가 다르다. 다만, 비율이 6:4 (데이터가 많은 경우 7:3 10만건, 딥러닝 1000만건)까지는 그냥 사용해도 된다. (데이터 : 데이터값 하나, 샘플 : row 한줄, 데이터셋 : 전체 데이터)

# 팩터를 숫자로 변환 해도 의미없는 숫자값이다. 행렬데이터로 바꿔야 함. 
# 행렬로 사용한다. 남성 1,여성 2 보다 0, 1 로 하는것이 낫다 


'data.frame':	8124 obs. of  23 variables:
 $ V1 : Factor w/ 2 levels "e","p": 2 1 1 2 1 1 1 1 2 1 ...
 $ V2 : Factor w/ 6 levels "b","c","f","k",..: 6 6 1 6 6 6 1 1 6 1 ...
 $ V3 : Factor w/ 4 levels "f","g","s","y": 3 3 3 4 3 4 3 4 4 3 ...
 $ V4 : Factor w/ 10 levels "b","c","e","g",..: 5 10 9 9 4 10 9 9 9 10 ...
 $ V5 : Factor w/ 2 levels "f","t": 2 2 2 2 1 2 2 2 2 2 ...
 $ V6 : Factor w/ 9 levels "a","c","f","l",..: 7 1 4 7 6 1 1 4 7 1 ...
 $ V7 : Factor w/ 2 levels "a","f": 2 2 2 2 2 2 2 2 2 2 ...
 $ V8 : Factor w/ 2 levels "c","w": 1 1 1 1 2 1 1 1 1 1 ...
 $ V9 : Factor w/ 2 levels "b","n": 2 1 1 2 1 1 1 1 2 1 ...
 $ V10: Factor w/ 12 levels "b","e","g","h",..: 5 5 6 6 5 6 3 6 8 3 ...
 $ V11: Factor w/ 2 levels "e","t": 1 1 1 1 2 1 1 1 1 1 ...
 $ V12: Factor w/ 5 levels "?","b","c","e",..: 4 3 3 4 4 3 3 3 4 3 ...
 $ V13: Factor w/ 4 levels "f","k","s","y": 3 3 3 3 3 3 3 3 3 3 ...
 $ V14: Factor w/ 4 levels "f","k","s","y": 3 3 3 3 3 3 3 3 3 3 ...
 $ V15: Factor w/ 9 levels "b","c"

 V1       V2       V3             V4       V5             V6       V7      
 e:4208   b: 452   f:2320   n      :2284   f:4748   n      :3528   a: 210  
 p:3916   c:   4   g:   4   g      :1840   t:3376   f      :2160   f:7914  
          f:3152   s:2556   e      :1500            s      : 576           
          k: 828   y:3244   y      :1072            y      : 576           
          s:  32            w      :1040            a      : 400           
          x:3656            b      : 168            l      : 400           
                            (Other): 220            (Other): 484           
 V8       V9            V10       V11      V12      V13      V14     
 c:6812   b:5612   b      :1728   e:3516   ?:2480   f: 552   f: 600  
 w:1312   n:2512   p      :1492   t:4608   b:3776   k:2372   k:2304  
                   w      :1202            c: 556   s:5176   s:4936  
                   n      :1048            e:1120   y:  24   y: 284  
                   g      : 752           

In [24]:
# 원핫인코딩 범주형 데이터~
library(mltools)
library(data.table)

head(one_hot(as.data.table(m)))


V1_e,V1_p,V2_b,V2_c,V2_f,V2_k,V2_s,V2_x,V3_f,V3_g,...,V22_s,V22_v,V22_y,V23_d,V23_g,V23_l,V23_m,V23_p,V23_u,V23_w
1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [22]:
dummy <- dummyVars(~., data = m)
mm <- data.frame(predict(dummy, newdata = m))

ERROR: Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels


In [ ]:
# 서울열린데이터광장에서 하고싶은 데이터로 주제 정해서 만들기
# 컬럼갯수 300개 이상
# 목요일에 시작

> 5/9 수업끗